In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objs as go
import plotly.io as pio
from dash import Dash, Input, Output, dcc, html
from pathlib import Path
import yfinance as yf
import logging

pio.renderers.default = 'notebook'

class PortfolioPerformance:
    def __init__(self, file=None, itvl='1d', lookback='10y'):
        self.logger = logging.getLogger(__name__)
        self.itvl = itvl
        self.lookback = lookback
        self.file = file
        self.folder = Path('cache')
        if not self.folder.exists() or not self.folder.is_dir():
            self.folder.mkdir()
        self.stocks = self._get_stocks()
        self.data = self._fetch_data()

    def _get_stocks(self):
        if self.file:
            try:
                with open(self.file, 'r') as f:
                    return [i.strip() for i in f.readlines()]
            except FileNotFoundError:
                self.logger.error('File not found.')
        return []

    def _fetch_data(self):
        if self.stocks:
            path = self.folder / 'ohlc.pkl'
            if not path.exists():
                self.logger.info(f'Fetching OHLC data for {len(self.stocks)} stocks...')
                df = yf.download(self.stocks, interval=self.itvl, period=self.lookback, progress=False, auto_adjust=True)
                df.to_pickle(path)
            else:
                df = pd.read_pickle(path)
            return df
        return pd.DataFrame()

    def pf_return(self, startdate=None):
        if startdate:
            startdate = pd.to_datetime(startdate)
        fig = go.Figure()
        fig.update_layout(
            template='plotly_dark',
            title=dict(
                text='📈 Portfolio Return',
                font=dict(size=24, family='Arial Black'),
                x=0.5,
                xanchor='center'
            ),
            font=dict(size=13, family='Arial'),
            yaxis=dict(
                title='Return (%)',
                gridcolor='rgba(128, 128, 128, 0.2)',
                showgrid=True
            ),
            xaxis=dict(
                title='Date',
                gridcolor='rgba(128, 128, 128, 0.2)',
                showgrid=True
            ),
            height=650,
            width=1200,
            hovermode='x unified',
            plot_bgcolor='rgba(17, 17, 17, 1)',
            paper_bgcolor='rgba(17, 17, 17, 1)',
            margin=dict(l=80, r=80, t=100, b=80)
        )
        if not self.data.empty:
            returns = self.data['Close'].pct_change().mean(axis=1).fillna(0.0) * 100
            if startdate is not None:
                returns = returns.loc[returns.index >= startdate]
            returns = returns.cumsum()
            ma_50 = ta.sma(returns, length=50).dropna()
            ma_200 = ta.sma(returns, length=200).dropna()

            fig.add_trace(go.Scatter(
                x=returns.index, y=returns,
                mode='lines',
                line=dict(color='#00D9FF', width=3, shape='spline'),
                name='Portfolio',
                fill='tozeroy',
                fillcolor='rgba(0, 217, 255, 0.15)',
                hovertemplate='<b>Return: %{y:.2f}%</b><extra></extra>'
            ))
            fig.add_trace(go.Scatter(
                x=ma_50.index, y=ma_50,
                mode='lines',
                line=dict(color='#FFD700', dash='dash', width=2),
                name='MA 50',
                hovertemplate='<b>MA50: %{y:.2f}%</b><extra></extra>'
            ))
            fig.add_trace(go.Scatter(
                x=ma_200.index, y=ma_200,
                mode='lines',
                line=dict(color='#FF6B9D', dash='dash', width=2),
                name='MA 200',
                hovertemplate='<b>MA200: %{y:.2f}%</b><extra></extra>'
            ))
        return fig

    def corr_matrix(self, startdate=None):
        if startdate:
            startdate = pd.to_datetime(startdate)
        fig = go.Figure()
        fig.update_layout(
            template='plotly_dark',
            title=dict(
                text='🔗 Return Correlation Matrix',
                font=dict(size=24, family='Arial Black'),
                x=0.5,
                xanchor='center'
            ),
            font=dict(size=11, family='Arial'),
            height=800,
            width=1200,
            plot_bgcolor='rgba(17, 17, 17, 1)',
            paper_bgcolor='rgba(17, 17, 17, 1)',
            margin=dict(l=100, r=100, t=100, b=100)
        )
        if not self.data.empty:
            returns = self.data['Close'].pct_change()
            if startdate:
                returns = returns.loc[returns.index >= startdate]
            corr = returns.corr()
            fig.add_trace(go.Heatmap(
                z=corr.values,
                x=corr.columns,
                y=corr.columns,
                colorscale='RdBu_r',
                zmid=0,
                zmin=-1,
                zmax=1,
                text=corr.values,
                texttemplate='%{text:.2f}',
                textfont=dict(size=9),
                colorbar=dict(
                    title='Correlation',
                    tickmode='linear',
                    tick0=-1,
                    dtick=0.5
                ),
                hovertemplate='%{x} vs %{y}<br>Correlation: %{z:.3f}<extra></extra>'
            ))
        return fig

    def atr(self, startdate=None):
        if startdate:
            startdate = pd.to_datetime(startdate)
        fig = go.Figure()
        fig.update_layout(
            template='plotly_dark',
            title=dict(
                text='💪 ATR - Volatility Indicator',
                font=dict(size=24, family='Arial Black'),
                x=0.5,
                xanchor='center'
            ),
            font=dict(size=13, family='Arial'),
            height=650,
            width=1200,
            xaxis=dict(
                title='Date',
                gridcolor='rgba(128, 128, 128, 0.2)',
                showgrid=True
            ),
            yaxis=dict(
                title='ATR(%)',
                gridcolor='rgba(128, 128, 128, 0.2)',
                showgrid=True
            ),
            hovermode='x unified',
            plot_bgcolor='rgba(17, 17, 17, 1)',
            paper_bgcolor='rgba(17, 17, 17, 1)',
            margin=dict(l=80, r=80, t=100, b=80)
        )
        if not self.data.empty:
            fdata = self.data.copy() if startdate is None else self.data.loc[self.data.index >= startdate].copy()
            atr = ta.atr(
                close=fdata['Close'].mean(axis=1),
                high=fdata['High'].mean(axis=1),
                low=fdata['Low'].mean(axis=1),
                length=14
            ).dropna()[:]/fdata['Close'].iloc[0].mean()
            
            mean_val = atr.mean()
            std_val = atr.std()
            
            fig.add_trace(go.Scatter(
                x=atr.index, y=atr,
                mode='lines',
                line=dict(color='#00FF9F', width=3, shape='spline'),
                name='ATR',
                fill='tozeroy',
                fillcolor='rgba(0, 255, 159, 0.1)',
                hovertemplate='<b>ATR: %{y:.2f}</b><extra></extra>'
            ))
            fig.add_hline(
                y=mean_val + std_val,
                line_dash='dot',
                line_color='#FF4444',
                annotation_text=f'Upper Band ({mean_val + std_val:.1f})',
                annotation_position='right'
            )
            fig.add_hline(
                y=mean_val - std_val,
                line_dash='dot',
                line_color='#FF4444',
                annotation_text=f'Lower Band ({mean_val - std_val:.1f})',
                annotation_position='right'
            )
            fig.add_hline(
                y=mean_val+4*std_val,
                line_dash='dash',
                line_color='rgba(255, 255, 255, 0.3)',
                annotation_text='High Volatility',
                annotation_position='left'
            )
        return fig

# ------------------- DASH APP ------------------- #

app = Dash(__name__)

app.layout = html.Div([
    # Header
    html.Div([
        html.H1(
            '📊 Portfolio Analytics Dashboard',
            style={
                'textAlign': 'center',
                'color': '#00D9FF',
                'fontFamily': 'Arial Black, sans-serif',
                'fontSize': '42px',
                'margin': '30px 0 10px 0',
                'textShadow': '2px 2px 4px rgba(0, 217, 255, 0.3)'
            }
        ),
        html.P(
            'Real-time portfolio performance tracking and analysis',
            style={
                'textAlign': 'center',
                'color': '#888',
                'fontFamily': 'Arial, sans-serif',
                'fontSize': '16px',
                'margin': '0 0 30px 0'
            }
        )
    ], style={
        'backgroundColor': 'rgba(17, 17, 17, 0.95)',
        'padding': '20px',
        'borderRadius': '15px',
        'margin': '20px auto',
        'maxWidth': '1200px',
        'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.3)'
    }),
    
    # Control Panel
    html.Div([
        html.Label(
            'Select Analysis:',
            style={
                'color': '#00D9FF',
                'fontFamily': 'Arial, sans-serif',
                'fontSize': '16px',
                'fontWeight': 'bold',
                'marginBottom': '10px',
                'display': 'block'
            }
        ),
        dcc.Dropdown(
            id='options',
            options=[
                {'label': '📈 Portfolio Return', 'value': 'return'},
                {'label': '🔗 Correlation Matrix', 'value': 'corr'},
                {'label': '💥 Volatility (ATR)', 'value': 'vol'}
            ],
            multi=False,
            value='return',
            clearable=False,
            style={
                'width': '100%',
                'fontFamily': 'Arial, sans-serif',
                'fontSize': '15px'
            }
        )
    ], style={
        'width': '400px',
        'margin': '0 auto 30px auto',
        'padding': '25px',
        'backgroundColor': 'rgba(17, 17, 17, 0.95)',
        'borderRadius': '12px',
        'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.3)',
        'border': '1px solid rgba(0, 217, 255, 0.2)'
    }),
    
    # Graph Container
    html.Div([
        dcc.Graph(
            id='graph',
            config={
                'displayModeBar': True,
                'displaylogo': False,
                'modeBarButtonsToRemove': ['pan2d', 'lasso2d', 'select2d']
            }
        )
    ], style={
        'margin': '0 auto',
        'maxWidth': '1250px',
        'padding': '20px',
        'backgroundColor': 'rgba(17, 17, 17, 0.95)',
        'borderRadius': '15px',
        'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.4)',
        'border': '1px solid rgba(0, 217, 255, 0.2)'
    }),
    
    # Footer
    html.Div([
        html.P(
            'Data updates automatically | Powered by yfinance & Plotly',
            style={
                'textAlign': 'center',
                'color': '#666',
                'fontSize': '12px',
                'margin': '0'
            }
        )
    ], style={
        'margin': '30px auto 20px auto',
        'maxWidth': '1200px'
    })
    
], style={
    'backgroundColor': '#0a0a0a',
    'minHeight': '100vh',
    'padding': '20px',
    'fontFamily': 'Arial, sans-serif'
})


@app.callback(
    Output('graph', 'figure'),
    Input('options', 'value')
)
def update_graph(value):
    pp = PortfolioPerformance('lists/filtered.txt')
    if value == 'return':
        figure = pp.pf_return()
    elif value == 'corr':
        figure = pp.corr_matrix()
    elif value == 'vol':
        figure = pp.atr()
    else:
        figure = go.Figure()
    
    return figure


if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    app.run(debug=True)